<a href="https://colab.research.google.com/github/golna001/visualwakewords/blob/master/MNIST_classification_small_wNoise_customLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install brevitas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.4/411.4 kB 8.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from brevitas.quant import IntBias  as BiasQuant
import brevitas.nn as qnn

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
# Load MNIST

transform = transforms.Compose(
    [transforms.ToTensor()])

batch_size = 16
nbits = 4

trainset = torchvision.datasets.MNIST(root='./dataMNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)


testset = torchvision.datasets.MNIST(root='./dataMNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

sample = next(iter(trainloader))
image, label = sample
print(image.shape)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

100%|██████████| 9912422/9912422 [00:00<00:00, 83005830.26it/s]


Extracting ./dataMNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./dataMNIST/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 76814010.03it/s]

Extracting ./dataMNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataMNIST/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23186214.77it/s]

Extracting ./dataMNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataMNIST/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 13946214.33it/s]


Extracting ./dataMNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataMNIST/MNIST/raw

torch.Size([16, 1, 28, 28])


In [ ]:
#@title train and test ratio
len(trainloader)/(len(trainloader)+len(testloader))

In [ ]:
#@title some data visualization
# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
class GaussianNoise(nn.Module):
    def __init__(self, std=None):
        super().__init__()
        self.std = std

    def forward(self, x, scale, kernel_size, input_channel):
        if self.std is not None and self.std != 0.0:
            noise = torch.randn(x.size()) * self.std * scale * kernel_size * np.sqrt(input_channel)
            x = x + noise
        return x

class LayerDiffStd(nn.Module):
    def __init__(self):
      super().__init__()
    def forward(self, x1, x2):
      return torch.std(torch.sub(x1, x2))

# My quantized model
class QNet(nn.Module):
    def __init__(self):
        # super(QNet, self).__init__()
        super().__init__()

        self.identity = qnn.QuantIdentity(bit_width=nbits,
                                          signed=False,
                                          return_quant_tensor=True)

        self.noise_module = GaussianNoise(None)

        self.conv1 = qnn.QuantConv2d(in_channels=1,
                                     out_channels=10,
                                     kernel_size=3,
                                     weight_bit_width=nbits,
                                     bias_quant=BiasQuant,
                                     bias_bit_width=nbits,
                                     return_quant_tensor=True)
        self.conv2 = qnn.QuantConv2d(in_channels=10,
                                     out_channels=16,
                                     kernel_size=3,
                                     weight_bit_width=nbits,
                                     bias_quant=BiasQuant,
                                     bias_bit_width=nbits,
                                     return_quant_tensor=True)

        self.maxpool = nn.MaxPool2d(2)

        self.relu = qnn.QuantReLU(bit_width=nbits, return_quant_tensor=True)

        self.dropout2 = nn.Dropout2d()
        self.dropout = nn.Dropout()

        self.fc1 = qnn.QuantLinear(in_features=400,
                                   out_features=50,
                                   bias=True,
                                   weight_bit_width=nbits,
                                   bias_quant=BiasQuant,
                                   bias_bit_width=nbits)
        self.fc2 = qnn.QuantLinear(in_features=50,
                                   out_features=10,
                                   bias=False,
                                   weight_bit_width=nbits,
                                   bias_quant=BiasQuant,
                                   bias_bit_width=nbits)
        self.lds = LayerDiffStd()

    def forward(self, x):
        out = self.identity(x)
        out = self.conv1(out)
        # print(out)
        out1 = self.noise_module(out,
                                out.scale.cpu().detach().numpy()[0][0][0][0],
                                self.conv1.kernel_size[0],
                                self.conv1.in_channels)
        # print(out1)
        # dif = self.lds(out1, out)
        # print(dif)
        out = self.maxpool(out1)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.noise_module(out,
                                out.scale.cpu().detach().numpy()[0][0][0][0],
                                self.conv2.kernel_size[0],
                                self.conv2.in_channels)
        out = self.dropout2(out)
        out = self.maxpool(out)
        out = self.relu(out)

        out = out.view(-1, 400)

        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)

        return F.log_softmax(out, dim = 1)

    def set_noise_level(self, noise_level):
        self.noise_module.std = noise_level

net = QNet()

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.7)
train_noise = 10
eval_noise = 10

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    net.set_noise_level(train_noise)
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.306
[1,   400] loss: 2.300
[1,   600] loss: 2.299
[1,   800] loss: 2.298
[1,  1000] loss: 2.294
[1,  1200] loss: 2.287
[1,  1400] loss: 2.283
[1,  1600] loss: 2.277
[1,  1800] loss: 2.264
[1,  2000] loss: 2.247
[1,  2200] loss: 2.242
[1,  2400] loss: 2.209
[1,  2600] loss: 2.174
[1,  2800] loss: 2.137
[1,  3000] loss: 2.069
[1,  3200] loss: 2.004
[1,  3400] loss: 1.905
[1,  3600] loss: 1.833
[2,   200] loss: 1.666
[2,   400] loss: 1.573
[2,   600] loss: 1.481
[2,   800] loss: 1.431
[2,  1000] loss: 1.362
[2,  1200] loss: 1.342
[2,  1400] loss: 1.253
[2,  1600] loss: 1.190
[2,  1800] loss: 1.154
[2,  2000] loss: 1.110
[2,  2200] loss: 1.098
[2,  2400] loss: 1.065
[2,  2600] loss: 1.021
[2,  2800] loss: 0.985
[2,  3000] loss: 0.981
[2,  3200] loss: 0.947
[2,  3400] loss: 0.925
[2,  3600] loss: 0.921
[3,   200] loss: 0.858
[3,   400] loss: 0.851
[3,   600] loss: 0.830
[3,   800] loss: 0.861
[3,  1000] loss: 0.820
[3,  1200] loss: 0.825
[3,  1400] loss: 0.822
[3,  1600] 

In [ ]:
#@title prediction visualization 1
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
#@title prediction visualization 2
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(batch_size)))

In [ ]:
# Accuracy of train
correct = 0
total = 0

net.set_noise_level(eval_noise)
net.eval()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in trainloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(trainloader)} train images: {100 * correct // total} %')

Accuracy of the network on the 3750 train images: 91 %


In [ ]:
# Accuracy of test
correct = 0
total = 0

net.set_noise_level(eval_noise)
net.eval()
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(testloader)} test images: {100 * correct // total} %')

<ipython-input-18-9d8734b27fc8>:15: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:363.)
  return torch.std(torch.sub(x1, x2))


tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0024)
tensor(0.0

KeyboardInterrupt: ignored

In [ ]:
print(eval_noise)

0.25


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: 0     is 99.1 %
Accuracy for class: 1     is 99.3 %
Accuracy for class: 2     is 96.4 %
Accuracy for class: 3     is 98.3 %
Accuracy for class: 4     is 98.1 %
Accuracy for class: 5     is 96.9 %
Accuracy for class: 6     is 97.7 %
Accuracy for class: 7     is 96.5 %
Accuracy for class: 8     is 93.9 %
Accuracy for class: 9     is 96.4 %


In [ ]:
model_name = 'MNIST_smallmodel'+'_'+str(nbits)+'bits.pkl'
torch.save(net.state_dict(), model_name)

In [ ]:
# model = torch.load('MNIST_model.pth')
model = torch.load('MNIST_smallmodel_4bits.pkl')
print(model)

# New Section